In [1]:
import sys
import pandas as pd
import numpy as np

from alerce.core import Alerce
client = Alerce()
import requests
import sqlalchemy as sa
import matplotlib.pyplot as plt

from IPython.core.display import HTML
display(HTML('<style>.container { width:100% !important; }</style>'))
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', None)
plt.style.use('default')
plt.rcParams['figure.figsize'] = (6., 4.)
plt.rcParams['figure.dpi'] = 72.

In [2]:
client = Alerce()


In [6]:
url = 'https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json'
params = requests.get(url).json()['params']

In [7]:
params

{'dbname': 'ztf',
 'user': 'alerceread',
 'host': '54.205.99.47',
 'password': 'w*C*u8AXZ4e%d+zv'}

In [8]:
engine = sa.create_engine('postgresql+psycopg2://' + params['user'] \
                          + ':' + params['password'] + '@' + params['host'] \
                          + '/' + params['dbname'])
conn = engine.connect()

In [11]:
names = ['ZTF18aabyvkm']
core_path = '/home/magdalena/Desktop/magister/SSL_DATASET/ZTF_SSL_Dataset'

query = '''
    SELECT
        oid, oid_catalog, dist
    FROM
        xmatch
    WHERE
        catid = 'allwise'
        AND oid IN (%s)
    ''' % (','.join(["'%s'" % oid for oid in names]))
   
df_xmatch = pd.read_sql_query(query, conn)
df_xmatch = df_xmatch.sort_values(by='dist').drop_duplicates('oid')

oid_catalog = [x for x in df_xmatch['oid_catalog']]

query = '''
    SELECT
        oid_catalog, w1mpro, w2mpro, w3mpro, w4mpro
    FROM
        allwise
    WHERE
        oid_catalog IN (%s)
    ''' % (','.join(["'%s'" % x for x in oid_catalog]))
   
df_wise = pd.read_sql_query(query, conn)

df_wise = pd.merge(df_xmatch, df_wise, on='oid_catalog', how='outer').set_index('oid')
del df_xmatch
   
query = '''
    SELECT
        oid, sgscore1, sgmag1, srmag1, distpsnr1
    FROM
        ps1_ztf
    WHERE
        oid IN (%s)
        ''' % (','.join(["'%s'" % oid for oid in names]))

df_ps1 = pd.read_sql_query(query, conn)
df_ps1 = df_ps1.drop_duplicates('oid').set_index('oid')

df_xmatch = pd.concat([df_wise, df_ps1], axis=1).reset_index()
del df_wise, df_ps1


df_xmatch.to_parquet(f'{core_path}/oids_new_tab.parquet')

In [12]:
df_xmatch.columns

Index(['oid', 'oid_catalog', 'dist', 'w1mpro', 'w2mpro', 'w3mpro', 'w4mpro',
       'sgscore1', 'sgmag1', 'srmag1', 'distpsnr1'],
      dtype='object')

In [13]:
query =  """
        select o.oid, 
        --from detections table
       d.ra, d.dec, d.mjd, d.fid, d.magpsf, d.sigmapsf, d.magpsf_corr, d.sigmapsf_corr_ext, d.isdiffpos, d.pid, d.candid,
        --from forced photometry table
        fp.mag, fp.e_mag, fp.mag_corr, fp.e_mag_corr_ext, fp.isdiffpos as isdiffpos_fp, fp.distnr ,
        --from ps table
        ps.sgscore1 , ps.sgmag1, ps.srmag1, ps.distpsnr1
        from  alerce.detection d 
        join  alerce."object" o on d.oid = o.oid
        join alerce.forced_photometry fp on fp.oid = o.oid
        join alerce.ps1_ztf ps on ps.oid = o.oid
        where o.oid = 'ZTF18aabyvkm'    
        """  
   
df_xmatch = pd.read_sql_query(query, conn)

In [14]:
df_xmatch.columns

Index(['oid', 'ra', 'dec', 'mjd', 'fid', 'magpsf', 'sigmapsf', 'magpsf_corr',
       'sigmapsf_corr_ext', 'isdiffpos', 'pid', 'candid', 'mag', 'e_mag',
       'mag_corr', 'e_mag_corr_ext', 'isdiffpos_fp', 'distnr', 'sgscore1',
       'sgmag1', 'srmag1', 'distpsnr1'],
      dtype='object')

In [15]:
df_xmatch[["candid",
            "tid",
            "mjd",
            "sid",
            "fid",
            "pid",
            "ra",
            "dec",
            "brightness",
            "e_brightness",
            "unit",]]

KeyError: "['tid', 'sid', 'brightness', 'e_brightness', 'unit'] not in index"

In [15]:
df_xmatch.drop_duplicates()

df_xmatch
df_xmatch.to_parquet(f'{core_path}/oids_new_detections.parquet')

In [104]:
query = """	
	EXPLAIN 
	SELECT oid, ndet, mjdstarthist, mjdendhist 
	FROM alerce."object" o
	WHERE mjdstarthist > {mjd_per_month[i]} AND mjdstarthist < {mjd_per_month[i+1]};

    """

In [107]:
conn.close()
